In [1]:
import os
import sys

parent_dir = os.path.abspath(os.path.join(os.pardir))
sys.path.append(parent_dir)

from personal_graph import database as db
from dotenv import load_dotenv # type: ignore

load_dotenv()
db_url = os.getenv("LIBSQL_URL")
auth_token = os.getenv("LIBSQL_AUTH_TOKEN")


In [2]:
# To initialze the database and establish a connection with tursodb
db.initialize(db_url, auth_token)

thread '<unnamed>' panicked at /root/.cargo/registry/src/index.crates.io-6f17d22bba15001f/libsql-0.3.1/src/hrana/hyper.rsTraceback (most recent call last):
  File "/home/anubhuti/Desktop/simple-graph-db/personal_graph/database.py", line 81, in initialize
:88:9:
there is no reactor running, must be called from the context of a Tokio 1.x runtime
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace
    return atomic(_init, db_url, auth_token)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
pyo3_runtime.PanicException: there is no reactor running, must be called from the context of a Tokio 1.x runtime


In [3]:
# find nodes like
def find_nodes_like(db_url, auth_token, label, threshold):
    return db.atomic(db.find_similar_nodes(label, threshold), db_url, auth_token)

label = "relative"
threshold = 0.9
find_nodes_like(db_url, auth_token, label, threshold)

Traceback (most recent call last):
  File "/tmp/ipykernel_16677/3809413473.py", line 3, in find_nodes_like
thread '<unnamed>' panicked at /root/.cargo/registry/src/index.crates.io-6f17d22bba15001f/libsql-0.3.1/src/hrana/hyper.rs:88:9:
there is no reactor running, must be called from the context of a Tokio 1.x runtime
pyo3_runtime.PanicException: there is no reactor running, must be called from the context of a Tokio 1.x runtime


[]

In [7]:
# merge by similarity

def merge_nodes(threshold, db_url, auth_token):
    db.atomic(db.pruning(threshold), db_url, auth_token)
    
threshold = 0.9
merge_nodes(threshold, db_url, auth_token)

Traceback (most recent call last):
thread '<unnamed>' panicked at /root/.cargo/registry/src/index.crates.io-6f17d22bba15001f/libsql-0.3.1/src/hrana/hyper.rs:  File "/tmp/ipykernel_16677/2195174852.py", line 4, in merge_nodes
88:9:
there is no reactor running, must be called from the context of a Tokio 1.x runtime
pyo3_runtime.PanicException: there is no reactor running, must be called from the context of a Tokio 1.x runtime


In [3]:
# To add a single node
def insert_single_node(db_url, auth_token, new_label, new_node, new_node_id):
    try:
        db.atomic(db.add_node(new_label, new_node, new_node_id), db_url, auth_token)
    except Exception as e:
        assert False
new_node = {'body': 'Jack is my cousin brothera and lives in America.'}
new_label = "relative"
new_node_id = 1

insert_single_node(db_url, auth_token, new_label, new_node, new_node_id)
    

Traceback (most recent call last):
  File "/tmp/ipykernel_12762/2862888417.py", line 4, in insert_single_node
pyo3_runtime.PanicException: there is no reactor running, must be called from the context of a Tokio 1.x runtime
thread '<unnamed>' panicked at /root/.cargo/registry/src/index.crates.io-6f17d22bba15001f/libsql-0.3.1/src/hrana/hyper.rs:88:9:
there is no reactor running, must be called from the context of a Tokio 1.x runtime


In [4]:
# To add multiple nodes
def bulk_insert_operations(db_url, auth_token, labels, nodes):
    ids = []
    bodies = []
    for id, body in nodes.items():
        ids.append(id)
        bodies.append(body)

    # bulk add and confirm
    db.atomic(db.add_nodes(bodies, labels, ids), db_url, auth_token)

new_nodes = {
    3 : {'name': 'Peri', 'age': '90'},
    4: {'name': 'Pema', 'age': '66'},
    5: {'name': "Ella", 'age': '45'}
}
new_labels = ["Retired Doctor", "Tuberculosis Patient", "Diabetes Pateint"]

bulk_insert_operations(db_url, auth_token, new_labels, new_nodes)

In [ ]:
# To find for a node
def find_a_node(db_url, auth_token, node):
            return db.atomic(db.find_node(node), db_url, auth_token)


find_a_node(db_url, auth_token, 4)

In [5]:
# For bulk upsert: Update and insert
def bulk_upsert(db_url, auth_token, labels, nodes):
    ids = []
    bodies = []
    for id, body in nodes.items():
        ids.append(id)
        bodies.append(body)

    db.atomic(db.upsert_nodes(bodies, labels, ids), db_url, auth_token)
nodes = {
    2 : {'name': 'Stanley', 'age': '30'},
    
    3 : {'name': 'Sheeran', 'type': ['singer', 'rich']}
}
labels = ["Lunch Box", "Pop Singer"]

bulk_upsert(db_url, auth_token, labels, nodes)
    

INSERTED EDGE


In [6]:
# For single upsert: Update and insert a node
def single_upsert(db_url, auth_token, label, body, id):
    
    db.atomic(db.upsert_node(id, label, body), db_url, auth_token)

body = {'name': 'Sheeran', 'type': ['singer', 'rich']}
label = "related as an"
id = 2
single_upsert(db_url, auth_token, label, body, id)

In [17]:
# Bulk connect and confirm
def bulk_node_connect(db_url, auth_token, labels, edges):
    sources = []
    targets = []
    attributes = []
    for src, tgts in edges.items():
        for target in tgts:
            tgt, label = target
            sources.append(src)
            targets.append(tgt)
            if label:
                attributes.append(label)
            else:
                attributes.append({})
    print(sources, targets, attributes)
        

    db.atomic(db.connect_many_nodes(
        sources, targets, label, attributes), db_url, auth_token)
    
edges = {
    2: [
        (1 , {'disease':'Diabetes'}),
        (2 , {'patient': 'Diabetes Symptoms'}),
    ]
}
edges_labels = ["Has", "is"]

bulk_node_connect(db_url, auth_token, edges_labels, edges)
    


Traceback (most recent call last):
  File "/tmp/ipykernel_12762/4126399405.py", line 18, in bulk_node_connect
thread '<unnamed>' panicked at /root/.cargo/registry/src/index.crates.io-6f17d22bba15001f/libsql-0.3.1/src/hrana/hyper.rs:88:9:
there is no reactor running, must be called from the context of a Tokio 1.x runtime
pyo3_runtime.PanicException: there is no reactor running, must be called from the context of a Tokio 1.x runtime


[2, 2] [1, 2] [{'disease': 'Diabetes'}, {'patient': 'Diabetes Symptoms'}]


In [6]:
# Single connect and confirm
def single_node_connect(db_url, auth_token, source, target, label, attribute):
    db.atomic(db.connect_nodes(
        source, target,  label, attribute), db_url, auth_token)
    
source = 5
target = 5
label = "treats"
attribute = {"person": "old man", "disease": "Asthma"}

single_node_connect(db_url, auth_token, source, target, label, attribute)

In [14]:
# To remove nodes
def remove_bulk_nodes(db_url, auth_token, ids):
        db.atomic(db.remove_nodes(ids), db_url, auth_token)

ids = [1, 4]

remove_bulk_nodes(db_url, auth_token, ids)


In [8]:
# To remove a single node
def remove_single_node(db_url, auth_token, id):
    db.atomic(db.remove_node(id), db_url, auth_token)

remove_single_node(db_url, auth_token, 5)

In [20]:
# To traverse the nodes
def traverse_nodes(db_url, auth_token, src, tgt):
    return db.traverse(db_url, auth_token, src, tgt)
traverse_nodes(db_url, auth_token, 2, 5)

Traceback (most recent call last):
  File "/home/anubhuti/Desktop/simple-graph-db/personal_graph/database.py", line 592, in traverse
thread '<unnamed>' panicked at /root/.cargo/registry/src/index.crates.io-6f17d22bba15001f/libsql-0.3.1/src/hrana/hyper.rs:88:9:
there is no reactor running, must be called from the context of a Tokio 1.x runtime
    return atomic(_traverse, db_url, auth_token)  # type: ignore
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
pyo3_runtime.PanicException: there is no reactor running, must be called from the context of a Tokio 1.x runtime


['2']

In [21]:
# To generate a query clause and find nodes
kv_name_like = db._generate_clause('name', predicate='LIKE')
print(db.atomic(db.find_nodes([kv_name_like], ('Pe%',)), db_url, auth_token))


Traceback (most recent call last):
thread '<unnamed>' panicked at /root/.cargo/registry/src/index.crates.io-6f17d22bba15001f/libsql-0.3.1/src/hrana/hyper.rs:88:  File "/tmp/ipykernel_12762/2893161704.py", line 3, in <module>
9:
there is no reactor running, must be called from the context of a Tokio 1.x runtime
pyo3_runtime.PanicException: there is no reactor running, must be called from the context of a Tokio 1.x runtime


[]


In [22]:
# Visualization
from personal_graph import visualizers
raw_path = "temp_path" +  "sample1.dot"
visualizers.graphviz_visualize(db_url, auth_token, raw_path, ['2'])

Traceback (most recent call last):
thread '<unnamed>' panicked at /root/.cargo/registry/src/index.crates.io-6f17d22bba15001f/libsql-0.3.1/src/hrana/hyper.rs:  File "/home/anubhuti/Desktop/simple-graph-db/personal_graph/visualizers.py", line 61, in graphviz_visualize
    for edge in db.atomic(connections(i), db_url, auth_token):  # type: ignore
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
pyo3_runtime.PanicException: there is no reactor running, must be called from the context of a Tokio 1.x runtime
88:9:
there is no reactor running, must be called from the context of a Tokio 1.x runtime
Traceback (most recent call last):
  File "/home/anubhuti/Desktop/simple-graph-db/personal_graph/visualizers.py", line 75, in graphviz_visualize
    node = db.atomic(db.find_node(i), db_url, auth_token)  # type: ignore
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
pyo3_runtime.PanicException: there is no reactor running, must be called from the context of a Tokio 1.x runtime

In [23]:
# Visualization with bodies

raw_path = "temp_path" +  "bodies.dot"
path_with_bodies = db.traverse(db_url, auth_token, 2, with_bodies=True)
visualizers.graphviz_visualize_bodies(raw_path, path_with_bodies)

Traceback (most recent call last):
  File "/home/anubhuti/Desktop/simple-graph-db/personal_graph/database.py", line 592, in traverse
thread '<unnamed>' panicked at /root/.cargo/registry/src/index.crates.io-6f17d22bba15001f/libsql-0.3.1/src/hrana/hyper.rs:88:9:
there is no reactor running, must be called from the context of a Tokio 1.x runtime
    return atomic(_traverse, db_url, auth_token)  # type: ignore
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
pyo3_runtime.PanicException: there is no reactor running, must be called from the context of a Tokio 1.x runtime


In [4]:
# vector search node
def vector_search_node(db_url, auth_token, data, k):
    try:
        db.atomic(db.vector_search_node(data, k), db_url, auth_token)
    except Exception as e:
        assert False
        
data = {"boxer": "has"}
k = 1
vector_search_node(db_url, auth_token, data, k)
    

Traceback (most recent call last):
thread '<unnamed>' panicked at /root/.cargo/registry/src/index.crates.io-6f17d22bba15001f/libsql-0.3.1/src/hrana/hyper.rs  File "/tmp/ipykernel_17363/1139132915.py", line 4, in vector_search_node
:88:9:
there is no reactor running, must be called from the context of a Tokio 1.x runtime
pyo3_runtime.PanicException: there is no reactor running, must be called from the context of a Tokio 1.x runtime


In [7]:
# vector search edge
def vector_search_edge(db_url, auth_token, data, k):
    try:
        db.atomic(db.vector_search_edge(data, k), db_url, auth_token)
    except Exception as e:
        assert False
        
data = {'subject': 'MES', 'type': ['person', 'Dr']}
k = 1
vector_search_edge(db_url, auth_token, data, k)
    

Traceback (most recent call last):
thread '<unnamed>' panicked at /root/.cargo/registry/src/index.crates.io-6f17d22bba15001f/libsql-0.3.1/src/hrana/hyper.rs:  File "/tmp/ipykernel_17363/1649741722.py", line 4, in vector_search_edge
88:9:
there is no reactor running, must be called from the context of a Tokio 1.x runtime
pyo3_runtime.PanicException: there is no reactor running, must be called from the context of a Tokio 1.x runtime
